# Aleasim Developer Guide
* Author : Yousif El-Wishahy
* Date: 2024-09-30
* Aleasim Design Document: https://docs.google.com/document/d/13W0VLFCf4jPUaaTYP6YeOesYwN5_omOXdhjFnd8Qf2M/edit
* Aleasim Wiki: https://wiki.aleasat.space/space-segment/aocs/aleasim/developer-guide 

# Aleasim Basics
The following guide and code cells assume you have followed the setup steps in the [developer guide wiki](https://wiki.aleasat.space/space-segment/aocs/aleasim/developer-guide)
Ensure that the interpreter of this Jupyter notebook is set to the aleasim poetry environment.

### Kernel and Model Creation

In [ ]:
#IMPORTS
#import the aleasim package
import alea.sim

#alternatively (reccomended), import specific modules from the package
from alea.sim.kernel.kernel import AleasimKernel

The aleasim.kernel.kernel.AleasimKernel class is responsible for startup/shutdown, connecting all the models, advancing the simulation time, and providing utility classes. The kernel manages simulation time and holds references to the Scheduler and FrameManager classes which handle simulation events and spatial transformations. 

Most importantly, the kernel holds a reference to “root” which is the root model of the simulation. The root model can be any class that implements AbstractModel, however, for simplicity the root model does nothing but contain other models. A class named AleasimRootModel implements AbstractModel.

In [ ]:
#A simulation that has no models

kernel = AleasimKernel()

We just instantiated a kernel. Let's add some models.

In [ ]:
#adding attitude dynamics to the kernel
from alea.sim.epa.attitude_dynamics import AttitudeDynamicsModel
from alea.sim.epa.orbit_dynamics import OrbitDynamicsModel

#first create the model object
#you must give it a name, and pass a reference of the kernel to it
adyn: AttitudeDynamicsModel = AttitudeDynamicsModel(kernel)

#at this point, the models __init__() and configure() functions should have been called.

#next you can add the model to the kernel
#you can optionally specify to create memory for it to store its state data in
kernel.add_model(adyn, create_shared_mem=True)

#now add orbital dynamics
odyn = OrbitDynamicsModel(kernel)
kernel.add_model(odyn, create_shared_mem=True)

kernel.step()

A couple of things just happened after we added an AttitudeDynamicsModel to the simulation:
* Attitude dynamics and orbital dynamics loaded configurations from files in `alea/sim/configuration/` , if you ever create a new model you should be adding configuration files in there
* Shared memory was created in attitude dynamics

FrameManager is a class that handles reference frame updates, it requires knowledge from orbit dynamics and attitude dynamics to get information for the transformations.

### Logging

Now that orbit dynamics is added, the frame manager is happily logging all the frame transformations between the different frames.

Sometimes logs can be excessive, logging level can be set on a model by model basis.

In [ ]:
import logging

#set logging levels for different components
kernel.frame_manager.logger.setLevel(logging.INFO)
adyn.logger.setLevel(logging.DEBUG)
odyn.logger.setLevel(logging.DEBUG)

kernel.step()

Or we can set the log level of all the models.

In [ ]:
kernel.set_log_level_all(logging.INFO)

### The Scheduler and Events

The Scheduler (aleasim.kernel.scheduler.Scheduler) is a generic interface for scheduling events. An instance of the scheduler is managed by the AleasimKernel and all children models register events to it.

Each update event that was logged previously is a registered event in the scheduler. When we call kernel.step() the schedule executes all events at that timestep. 

An event in this context is a Dataclass containing some metadata: (time, priority, sequence, action, period, argument ).
time - simulator time to execute event at
priority - events scheduled for the same time will be executed in the order of their priority
sequence - A continually increasing sequence number that  separates events if time and priority are equal
period - Time offset for next periodic event, if -1 will assume not periodic.
action - Executing the event means executing action(*argument, **kwargs)
argument -  a sequence holding the positional arguments for the action.

At its core the scheduler maintains a queue of ordered events (based on time -> priority -> sequence number). The key point is that “Executing the event means executing action(*argument, **kwargs)”. The general way that events should be used in the simulation is to schedule periodic events to the update function of the model, or schedule one-time events when needed based on logic. The scheduler is permitted to run while events can be added to the queue. For now this scheduler is designed to run in a single-threaded, single-process manner, but in general can support multiple threads.

Lets look at the pending events!

In [ ]:
#print the scheduler event queue
print(f'length of event queue is {len(kernel.scheduler.queue)}')
print(kernel.scheduler.queue)

You should see that the next event in the queue `Event(time=6, priority=10, is_periodic=True, period=1, action=<bound method FrameManager.update_frames of <aleasim.kernel.frame_manager.FrameManager object at 0xsomeRamLocation>>`

But we can add more events:


In [ ]:
def test_event_function():
    print("test event!")

#schedule an function event to execute immediately with a priority of 1, and only execute once
kernel.schedule_event(delay=0, priority=1, action=test_event_function)

#we should see a printout now
kernel.step()


### Simulation Time

In [ ]:
#we can call kernel.advance to advance simulation time for x seconds

#advance for 0.1 seconds
kernel.advance(0.1)

#simulation timestep
print(f'simulation timestep is {kernel.timestep} s')

#simulation seconds since start
print(f'simualtion time in seconds is {kernel.time}')

#the float time has floating point error
#internally, integers are used
print(f'simulation integer time is {kernel._t_n} (multiples of timestep)')

#simulation date in utc
print(f'simulation date is {kernel.date}')

#skyfield epoch time is used by the simulation for computing planet locations
print(f'simulation skyfield epoch time is {kernel.skyfield_time}')


## Model Development Example

Let's take a quick look at creating a basic dummy magnetic sensor model.

We want it to do 2 things
* power on/off and consume power
* report magnetic field value with some constant bias

Let's grab a section from the design doc:

### Abstract Model
`alea/sim/kernel/generic/abstract_model.py`
The core Aleasim model is the AbstractModel which has some generic functionalities that will be inherited by all models. This functionality includes generic functions for logging, configuring and connecting models, updating the model state, and containing children models. All AbstractModels also hold a reference to the kernel and scheduler.


Methods that all inheriting classes can choose to override:
```python

    def connect(self):
        """
        Connect models to each other, and obtain references to attributes of other models
        this function may return an error if the parent is undefined or if a model is missing
        from the tree.
        
        This method will be invoked as a kernel event at simulation time 0 before any other events
        on this model.
        """
        pass

    def start(self):
        """
        Start any processes that require the connections established in self.connect() and must also
        start before the main simulation.
        
        This method will be invoked as a kernel event at simulation time 0 after all connect events
        but before any other events.
        """
        pass
```
If the these methods are not overriden they will do nothing (`pass` will be called).

### Adding additional features to models

Additional features can be added to a model by inheriting other classes, this is multiple inheritace in OOP and the classes are termed *mixin*s. [Mixins are a special type of multiple inheritance](https://stackoverflow.com/questions/533631/what-is-a-mixin-and-why-is-it-useful).

There are several mixin classes available, each adding a different feature.

Each mixin may add class attributes, properties, and methods like any other class. Mixins may also add abstract methods for the inheriting model to implement.

[Method resolution order](https://stackoverflow.com/questions/1848474/method-resolution-order-mro-in-new-style-classes) for multiple inheritance in Python is from left-right in the simplest case, though there are more complex cases. Mixins are developed to be order independant because they pass on `**kwargs`, however the `AbstractModel` must be the base class.

#### Passing on constructor arguments with multiple inheritance
Because mixins pass on `**kwargs**, the constructor of the inheriting class must specify the names of the arguments.

For example

```python
class ReactionWheelModel(Configurable[ReactionWheelConfig], SharedMemoryModelInterface, PoweredUnitModel, DynamicModel, AbstractModel):
    """
    Reaction Wheel Model.
    
    Initialize with spin_axis_body which is the spin axis in the spacecraft BODY frame.
    """
    
    def __init__(self, sim_kernel: AleasimKernel, name: str, spin_axis_body: np.ndarray,  cfg: str | Path | dict | ReactionWheelConfig = "reaction_wheel") -> None:
        super().__init__(name=name, sim_kernel=sim_kernel, cfg=cfg, cfg_cls=ReactionWheelConfig)
        
        ...
```


#### `DynamicModel` class
`alea/sim/kernel/generic/dynamic_model.py`

Inherits AbstractModel, contains functionality for numerical integration of differential equations governing the dynamical state of the model.

The inheiting model must implement the state update function which returns the derivative of the state vector.

```python
    @abc.abstractmethod
    def state_update_fcn(self, t:np.ndarray|float, x:np.ndarray | float, u:np.ndarray | float, update_params: dict = None) -> np.ndarray | float:
        """State update function that returns the state derivative at time t. Must be implemented in inheriting class.

        Args:
            t:
                System time.

            x:
                System state.

            u:
                System input.

            update_params (optional):
                Any additional parameters needed to calculate the state derivative in a dictionary.

        Returns:
            Time derivative of system state.
        """
        raise NotImplementedError()
```

#### `PoweredUnitModel` class
`alea/sim/kernel/generic/powered_unit_model.py`

Simple powered unit that turns on and off and consumes power
The power calculation tracks a time varying current with a constant voltage source.
Alternatively the load can be resistive and only have a constant power.

The following methods/properties must be implemented by inheriting classes
```python
    @abstractmethod
    def calculate_active_power_usage(self) -> float:
        raise NotImplementedError()
    
    @property
    @abstractmethod
    def current(self) -> float:
        raise NotImplementedError()
```

#### `Configurable` class
`alea/sim/configuration/__init__.py`

Loads a configuration dataclass from json formatted `.cfg` files in `alea/sim/configuration` directory.
A configuration dataclass must be specified.

#### `SharedMemoryModelInterface` class
`alea/sim/kernel/kernel.py`

Supports saving an aribtrarily sized model state to a shared memory array.

```python
    @property
    @abc.abstractmethod
    def saved_state_size(self) -> int:
        """
        This property must be implemented by inheriting subclass.
        length of a single chunk for model datastorage, typically composed of float64s
        this information is relevant to shared memory
        """
        raise NotImplementedError()

    @property
    @abc.abstractmethod
    def saved_state_element_names(self) -> list[str]:
        """
        This property must be implemented by inheriting subclass.
        list of names for each element in the saved state
        """
        raise NotImplementedError()
```

#### `TimeCachedModel` class
`alea/sim/kernel/time_cached.py`

A mixin that provides a mechanism to calculate variables on-the-fly and cache their
results, so that not all values have to be calculated in a periodic "update" function.

Subclasses should call self.invalidate_cache(current_time) and pass the current simulation
time whenever the model is requested to update.

self.invalidate_cache(current_time) should be called before assigning to any cached
properties to ensure the new property values are associated with the latest timestamp.


In [ ]:
#based on the available models, we want
# 1. inherit powered unit, and abstract model
# 2. inherit SharedMemoryModelInterface to save our sensor data
# 3. obtain magnetic field data somehow
# 4. write a sensor output function

#first import abstract model and other model feature clases
from alea.sim.kernel.generic.abstract_model import AbstractModel
from alea.sim.kernel.generic.powered_unit_model import PoweredUnitModel
from alea.sim.kernel.kernel import SharedMemoryModelInterface

#magnetic field model
from alea.sim.epa.earth_magnetic_field import EarthMagneticFieldModel
from alea.sim.kernel.kernel import AleasimKernel

#frame imports
from alea.sim.kernel.frames import ReferenceFrame
from alea.sim.kernel.frame_manager import FrameManager

#numpy for vector math
import numpy as np

from math import degrees
class MagSensorModel(SharedMemoryModelInterface, PoweredUnitModel, AbstractModel):
    """
    Simple mag sensor that returns the local magnetic field to the spacecraft with some bias added to it.
    """   
    
    def __init__(self, name: str, sim_kernel: AleasimKernel, power_usage: float = 0, v_in:float = 3.3) -> None:
        super().__init__(name=name, sim_kernel=sim_kernel)
        
        #sensor frame
        #use the sim body frame as the sensor frame
        self._frame_manager: FrameManager = self.kernel._frame_manager
        self._frame = self._frame_manager.body_frame
        
        self._constant_bias: np.ndarray = np.ones(3)*2.73e-8
        
        # magnetic field ground truth model
        # remember to grab models in connect()
        self._magm: EarthMagneticFieldModel = None
        
        self._power_usage = power_usage
        self._v_in = v_in

    def connect(self):
        #grab the magnetic field model
        self._magm = self.kernel.get_model(EarthMagneticFieldModel)

    @property
    def current(self) -> float:
        return self.calculate_active_power_usage()/self._v_in

    def calculate_active_power_usage(self) -> float:
        if self.is_powered_on:
            return self._power_usage 
        else:
            return 0.0

    @property
    def saved_state_size(self) -> int:
        return 3

    @property
    def saved_state_element_names(self) -> list[str]:
        return ['magx', 'magy', 'magz']
    
    #custom function
    def measure(self) -> np.ndarray:

        # get the magnetic field vector in NED frame
        mag_ned = self._magm.mag_field_vector_ned
        
        # transform from ned to desired frame
        mag_sens = self._frame.transform_vector_from_frame(mag_ned, self._frame_manager.ned_frame)
        
        #add a contant bias to it
        mag_sens += self._constant_bias
        
        self.save_chunk_to_memory(mag_sens)
        
        return mag_sens

We now have a basic sensor class `MagSensorModel` lets try to add it to the simulation.

In [ ]:
#first create a new simulation
kernel.kill()
del kernel
kernel = AleasimKernel()
adyn: AttitudeDynamicsModel = AttitudeDynamicsModel(kernel)
kernel.add_model(adyn, create_shared_mem=True)
odyn = OrbitDynamicsModel(kernel)
kernel.add_model(odyn, create_shared_mem=True)

magsense = MagSensorModel('mag_sensor', kernel, power_usage=1e-3)

kernel.add_model(magsense)

Let's try to step the simulation and measure the sensor.

In [ ]:
try:
    kernel.step()
except Exception as errmsg:
    print(f'error: {errmsg}\n')

Aha! We forgot to do one thing... `MagSensorModel` searches for `EarthMagneticFieldModel` but the simulation has no such model ... yet.

In [ ]:
#add magnetic field model
magm = EarthMagneticFieldModel(kernel)
kernel.add_model(magm)

#force call connect again to find the magnetic model
magsense.connect()

#now try stepping
kernel.step()

#print a measurement
print(f'**********\nmag sensor measurement: {magsense.measure()} nT\n**********')

In [ ]:
print(f'spacecraft position in eci [m] is {odyn.position_eci}')

Note that we haven't used any of the power logic yet. If we like we could allow it so that the measurement is only valid if the unit is powered on. I leave at as an exercise for the reader. For now here's an example of the inherited power functions.

In [ ]:
print(magsense.powered_state)

magsense.power_on()

print(magsense.calculate_active_power_usage())
print(magsense._power_on_time)

magsense.measure()

print(magsense.state_array)

## Plotting and Analysis

Now that we have created shared memory, lets look at some attitude date.

The attitude dynamics model describes the state vector which it saves to shared memory as 

```
    The spacecraft attitude state is is a 7 element vector composed of
    - quaternion elements (4) (rotation of body frame w.r.t inertial frame)
    - body angular rates (3) (angular velocity of body frame w.r.t inertial frame)
```

This gives us enough information to print some elements.

In [ ]:
import matplotlib.pyplot as plt
logging.disable() #too many logs from matplotlib

#first run the kernel for a few seconds
#recall the timestep is 0.001
kernel.advance(2)

#plot only 1000 values
n = 1000

objs = plt.plot(adyn.state_array[:n,0:4])
plt.title("Quaternion Elements")
plt.legend(iter(objs), ('q0', 'q1', 'q2', 'q3'))
plt.show()

objs = plt.plot(adyn.state_array[:n,4:7])
plt.title("Angular Rates")
plt.legend(iter(objs), ('w1', 'w2', 'w3'))
plt.show()

That was boring. The state vector is just constant. Let's add some angular velocity.

In [ ]:
#set a new angular velocity of body frame relative to inertial
new_state = adyn._state
new_state[4:7] = [30, 0, -10] #rad/s
adyn.set_state(new_state)

#start index
nstart = adyn.arr_size

#first run the kernel for 1 second
#recall the timestep is 0.001
kernel.advance(1)

#plot only 1000 values
n = nstart + 1000

objs = plt.plot(adyn.state_array[nstart:n,0:4])
plt.title("Quaternion Elements")
plt.legend(iter(objs), ('q0', 'q1', 'q2', 'q3'))
plt.show()

objs = plt.plot(adyn.state_array[nstart:n,4:7])
plt.title("Angular Rates")
plt.legend(iter(objs), ('w1', 'w2', 'w3'))
plt.show()

## Environment Vector: Sun Vector in ECI and BODY frames

In [ ]:
#the sun vector is obtained from orbital information , so it is contained in the orbit dynamics model

#odyn returns the sun vector in units of au

#get sun vector in eci frame
sun_vec_eci: np.ndarray = odyn.sun_vector
print('sun vec eci', sun_vec_eci)

#convert to body frame (to use as a sensor input for example)
sun_vec_body: np.ndarray = kernel.body_frame.transform_vector_from_frame(sun_vec_eci, kernel.eci_frame)
print('sun vec body', sun_vec_body)

#they should have the same magnitude
print(f'|sun_vec_eci| == |sun_vec_body| is {np.linalg.norm(sun_vec_body) == np.linalg.norm(sun_vec_eci)}')

In [ ]:
#cleanup
kernel.kill()
del kernel

# Attitude Control Demo

In this demo a single model is added to the kernel, this is the `AleasimRootModel` which creates its own submodels

```python
class AleasimRootModel(RootModel):
    """
    Root model of a useful aleasim.
    
    Instantiates the following children models:
        - EPAModel
        - Spacecraft
    """

    def __init__(self, sim_kernel: AleasimKernel) -> None:
        super().__init__('aleasim_root', sim_kernel)
        
        self.epa = EPAModel(sim_kernel)
        self.spacecraft = Spacecraft(sim_kernel)
        
        sim_kernel.add_model(self.epa, parent=self)
        sim_kernel.add_model(self.spacecraft, parent=self)
```

The spacecraft and epa models create their own sub models as well!

In [ ]:
# ==============================================================================
# Import Root Model
# ==============================================================================

from alea.sim.kernel.root_models import AleasimRootModel

# ==============================================================================
# Setup New Kernel
# ==============================================================================

kernel = AleasimKernel(dt=0.01, date=2024.2)
root_model = AleasimRootModel(kernel)
kernel.add_model(root_model)

## Aleasim Model Tree

In the next block, let us take a look at what the model tree looks like:

In [ ]:
# we will use the json module to printed the nested dictionaries of model names
# https://stackoverflow.com/questions/3229419/how-to-pretty-print-nested-dictionaries

import json

nested = kernel._root.get_children_model_names_nested()
print(json.dumps(nested, sort_keys=True, indent=4))

In [ ]:
# print type info as well
nested = kernel._root.get_children_model_names_nested(include_types=True)
print(json.dumps(nested, sort_keys=True, indent=4))

## Run the Control Demo

In [ ]:
from alea.sim.spacecraft.spacecraft import Spacecraft
from alea.sim.spacecraft.eps.power_system import PowerSystemModel
from alea.sim.spacecraft.eps.solar_panel import SolarPanelModel

# ==============================================================================
# Grab models from the kernel
# ==============================================================================

adyn: AttitudeDynamicsModel = kernel.get_model(AttitudeDynamicsModel)
odyn: OrbitDynamicsModel = kernel.get_model(OrbitDynamicsModel)
sc: Spacecraft = kernel.get_model(Spacecraft)

rwx, rwy, rwz = sc.aocs._rws
mtqx, mtqy, mtqz =  sc.aocs._mtqs

mag_sens = sc.aocs._mag_sens
sun_sens = sc.aocs._sun_sens
gyro = sc.aocs._gyro_sens

power_sys = PowerSystemModel(kernel)

solar_panels: list[SolarPanelModel] = power_sys.solar_panels
eps = power_sys.eps

# ==============================================================================
# Configure
# ==============================================================================

eps._force_state_of_charge(50)

adyn.set_state(np.array([0.1,0.5,0.1,0,0,0,0.0]))

kernel.advance_n(2)
sc.aocs.set_mode(sc.aocs.AOCSMode.POINTING_SUN_A)

# ==============================================================================
# Run the control demo for 10 seconds
# ==============================================================================

kernel.advance(100)

## Plots

Lets look at the state of some of the models throughout the demo.

### Attitude Dynamics and AOCS Performance Plots

In [ ]:
aocs = sc.aocs

objs = plt.plot(aocs.time_array, aocs.state_array[:,8:11])
plt.legend(iter(objs), aocs.saved_state_element_names[8:11])
plt.title("Absolute Knowledge Error")
plt.xlabel('Time (s)')
plt.ylabel('degrees')
plt.show()

objs = plt.plot(aocs.time_array, aocs.state_array[:,11:14])
plt.legend(iter(objs), aocs.saved_state_element_names[11:14])
plt.title("Absolute Pointing Error")
plt.xlabel('Time (s)')
plt.ylabel('degrees')
plt.show()

fig, axs = plt.subplots(4)
for i in range(4):
    ax = axs[i]
    ax.plot(aocs.time_array, aocs.state_array[:,i], label=f'q{i}_target', linestyle='dashed')
    ax.plot(aocs.time_array, aocs.state_array[:,4+i], label=f'q{i}_estimated', linestyle='dashed')
    ax.plot(adyn.time_array, adyn.state_array[:,i], label=f'q{i}_true',linewidth=3)
    ax.legend()
plt.suptitle("Estimated and Real Quaternion Elements")
plt.xlabel('Time (s)')
plt.show()

objs = plt.plot(adyn.time_array, adyn.state_array[:,4:7])
plt.title("Angular Rates")
plt.legend(iter(objs), ('w1', 'w2', 'w3'))
plt.ylabel('rad/s')
plt.xlabel('Time (s)')
plt.show()


objs = plt.plot(adyn.time_array, adyn.state_array[:,7:10])
plt.title("Angular Acceleration")
plt.legend(iter(objs), tuple(adyn.saved_state_element_names[7:10]))
plt.ylabel('rad/s^2')
plt.xlabel('Time (s)')
plt.show()


objs = plt.plot(adyn.time_array, adyn.state_array[:,10:])
plt.title("Disturbance Torques")
plt.legend(iter(objs), tuple(adyn.saved_state_element_names[10:13]))
plt.ylabel('Nm')
plt.xlabel('Time (s)')
plt.show()


for rw, id in zip(aocs._rws, ['x', 'y', 'z']):
    plt.plot(rw.time_array, rw.state_array[:,0], label=id)
plt.title("Reaction Wheel Torques")
plt.legend()
plt.ylabel('Nm')
plt.xlabel('Time (s)')
plt.show()


for mtq, id in zip(aocs._mtqs, ['x', 'y', 'z']):
    plt.plot(mtq.time_array, mtq.state_array[:,0], label=id)
plt.title("Mtq Dipole Moments")
plt.legend()
plt.ylabel('Am^2')
plt.xlabel('Time (s)')
plt.show()

sensors = [gyro, sun_sens, mag_sens]
for sens in sensors:
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    for i in range(3):
        ax1.plot(sens.time_array, sens.state_array[:,i], label=sens.saved_state_element_names[i])
        ax2.plot(sens.time_array, sens.state_array[:,i+sens.axes], label=sens.saved_state_element_names[i+sens.axes])
        ax1.legend()
        ax2.legend()
    plt.suptitle(f'{sens.name} Measurements vs. Ground Truth')
    plt.xlabel('Time (s)')
    plt.show()

### Actuator Plots

In [ ]:
##['torque_rw', 'torque_cmd_rw', 'torque_cmd_no_delay_rw', 'acceleration_rw', 'friction_torque_rw', 'velocity_rw', 'momentum_rw', 'current_rw', 'power_rw']
for rw in aocs._rws:
    axs = plt.plot(rw.time_array,rw.state_array[:, 0], label=rw.saved_state_element_names[0]+ rw.name)
plt.title("Wheel Torque")
plt.ylabel('Torque [Nm]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for rw in aocs._rws:
    axs = plt.plot(rw.time_array,rw.state_array[:, 1], label=rw.saved_state_element_names[1]+ rw.name)
plt.title("Wheel Torque Command")
plt.ylabel('Torque [Nm]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()


for rw in aocs._rws:
    plt.plot(rw.time_array,rw.state_array[:,3], label=rw.saved_state_element_names[3]+ rw.name)
plt.title("Wheel Acceleration")
plt.ylabel('Accel [rad/s^2]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for rw in aocs._rws:
    plt.plot(rw.time_array,rw.state_array[:,4], label=rw.saved_state_element_names[4]+ rw.name)
plt.title("Friction Torque")
plt.ylabel('Torque [Nm]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for rw in aocs._rws:
    plt.plot(rw.time_array,rw.state_array[:,5], label=rw.saved_state_element_names[5]+ rw.name)
plt.title("Wheel Velocity")
plt.ylabel('Velocity [rad/s]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for rw in aocs._rws:
    plt.plot(rw.time_array,rw.state_array[:,6], label=rw.saved_state_element_names[6]+ rw.name)
plt.title("Wheel Momentum")
plt.ylabel('Momentum [kg m^2 rad/s]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for rw in aocs._rws:
    plt.plot(rw.time_array,rw.state_array[:,7], label=rw.saved_state_element_names[7] + rw.name)
plt.title("Wheel Current Draw")
plt.ylabel('Current [A]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for mtq in aocs._mtqs:
    plt.plot(mtq.time_array, mtq.state_array[:,0], label=mtq.saved_state_element_names[0] + mtq.name)
plt.title("Magnetorquer Moments")
plt.ylabel('Moment [Am^2]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for mtq in aocs._mtqs:
    plt.plot(mtq.time_array, mtq.state_array[:,4], label=mtq.saved_state_element_names[4] + mtq.name)
plt.title("Magnetorquer Current Draw")
plt.ylabel('Moment [Am^2]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

### Orbit Plots

In [ ]:
print(f'data elements: {odyn.saved_state_element_names}')
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(odyn.state_array[:,0],odyn.state_array[:,1], odyn.state_array[:,2])
ax.set_xlabel('X ECI [m]')
ax.set_ylabel('Y ECI [m]')
ax.set_zlabel('Z ECI [m]')
plt.show()

plt.title("ECI Position")
objs = plt.plot(odyn.time_array, odyn.state_array[:,0:3])
plt.legend(iter(objs), tuple(odyn.saved_state_element_names[0:3]))
plt.xlabel('Time (s)')
plt.show()

plt.title("ECI Velocity")
objs = plt.plot(odyn.time_array, odyn.state_array[:,3:6])
plt.legend(iter(objs), tuple(odyn.saved_state_element_names[3:6]))
plt.xlabel('Time (s)')
plt.show()

plt.title("Lon, Lat")
objs = plt.plot(odyn.time_array, odyn.state_array[:,6:8])
plt.legend(iter(objs), tuple(odyn.saved_state_element_names[6:8]))
plt.xlabel('Time (s)')
plt.show()

plt.title("WGS84 Altitude")
plt.plot(odyn.time_array, odyn.state_array[:,8])
plt.xlabel('Time (s)')
plt.show()

### Energy Plots

In [ ]:
#solar panel states - ['pwr_gen','energy_gen_total']
for panel in solar_panels:
    plt.plot(panel.time_array, panel.state_array[:,0], label=panel.name)
plt.legend()
plt.title('Power Generated per Panel')
plt.ylabel('Power (W)')
plt.xlabel('Time (s)')
plt.show()

#eps states: ['battery_soc_percent', 'battery_soc_Ah', 'watt_hour_estimate', 'solar_current', 'load_current', 'eps_voltage', 'power_net', 'power_in', 'power_out']
plt.plot(eps.time_array, eps.state_array[:,0])
plt.title('Eps State of Charge %')
plt.ylabel('%')
plt.xlabel('Time (s)')
plt.show()

plt.plot(eps.time_array, eps.state_array[:,1])
plt.title('Eps State of Charge Amp Hours')
plt.ylabel('Ah')
plt.xlabel('Time (s)')
plt.show()

plt.plot(eps.time_array, eps.state_array[:,2])
plt.title('Eps Watt Hours (inst.)')
plt.ylabel('Ah')
plt.xlabel('Time (s)')
plt.show()

objs = plt.plot(eps.time_array, eps.state_array[:,3:5])
plt.legend(iter(objs), tuple(eps.saved_state_element_names[3:5]))
plt.title('Currents')
plt.ylabel('A')
plt.xlabel('Time (s)')
plt.show()

plt.plot(eps.time_array, eps.state_array[:,5])
plt.title('Eps voltage')
plt.ylabel('V')
plt.xlabel('Time (s)')
plt.show()

for i in range(6,9):
    plt.plot(eps.time_array, eps.state_array[:, i], label=eps.saved_state_element_names[i])
mean_power_in = eps.state_array[:, 7].mean()
mean_power_out = eps.state_array[:, 8].mean()
plt.hlines(mean_power_in, eps.time_array[0], eps.time_array[-1], label='Average Power In', colors='g', linestyles='--')
plt.hlines(mean_power_out, eps.time_array[0], eps.time_array[-1], label='Average Power Out', colors='r', linestyles='--')
plt.title("EPS Power")
plt.legend()
plt.ylabel('W')
plt.xlabel('Time (s)')
plt.show()